# Locate Address Object

## Imports

We need to import several libraries to accomplish our task

In [1]:
# standard library imports
import os

# third party library imports
import argparse
import pandas as pd
from tabulate import tabulate
from dotenv import load_dotenv


# Palo Alto Networks imports
from panos.panorama import Panorama, DeviceGroup
from panos.objects import AddressObject, AddressGroup

## Set up our connection to Panorama

We will leverage `python-dotenv` to load environment variables into our script. After the basic credientials have been loaded into a Panorama object, we will save that object as `pan`

In [2]:
load_dotenv(".env")
PANURL = os.environ.get("PANURL", "panorama.lab")
PANUSER = os.environ.get("PANUSER", "automation")
PANPASS = os.environ.get("PANPASS", "mysecretpassword")
pan = Panorama(PANURL, PANUSER, PANPASS)

## Main function

Here is where our program lives


In [3]:
def grab_config():
    """
    Description: collect configuration objects from Panorama.
    Workflow: 
        1. Pull in various components of a Panorama configuration.
            - Device Groups
            - Address Groups
            - Address Objects
        2. Identify all address objects and append them to "address_objects"
        3. Identify all address groups and append them to "address_groups"
        4. Loop over device groups and perform steps 2 & 3 again.
    Return:
        - name: address_groups
          type: tuple
        - name: address_objects
          type: tuple
    """


    # load Panorama configuration objects
    pan_address_objects = AddressObject.refreshall(pan)
    pan_address_groups = AddressGroup.refreshall(pan)

    # create empty placeholders
    address_objects = []
    address_groups = []
    
    # append shared config address objects
    for each in pan_address_objects:
        address_objects.append(("Shared", each.name, each.value, each.type))

    # append shared config address groups
    for each in pan_address_groups:
        if each.static_value:
            if not each.description:
                each.description = ""
            address_groups.append(("Shared", each.name, each.description, each.static_value))    

    # pull down list of device groups
    device_groups = DeviceGroup.refreshall(pan)

    # loop over device groups and perform the same actions
    for dg in device_groups:

        # start with address objects
        dg_address_objects = AddressObject.refreshall(dg)
        for each in dg_address_objects:
            address_objects.append((dg.name, each.name, each.value, each.type))

        # finish with address groups
        dg_address_groups = AddressGroup.refreshall(dg)
        for each in dg_address_groups:
            if each.static_value:
                if not each.description:
                    each.description = ""
                address_groups.append((dg.name, each.name, each.description, each.static_value))    
    
    # return our address_groups and address_objects to the main function
    return address_groups, address_objects


In [4]:
def main(address_groups, address_objects, search):
    # create a placeholder for our potential match
    match = {}

    # loop over the address objects
    for each in address_objects:

        # if there is a positive match, then update our match object
        if search in each:
            match["source"] = each[0]
            match["name"] = each[1]

    # create a placeholder for our potential groups
    associated_groups = []

    # loop over the address groups
    for each in address_groups:

        # if there is a positive match, then update our associated_groups object
        if match["name"] in each[3]:
            associated_groups.append(each)
    
    # let's finally loop over our associated_groups object to see if a group is nested
    for each in associated_groups:

        # loop over our address_groups object again, looking to find a match
        for group in address_groups:
            
            # append when we see the name of our address group matched in another address group
            if each[1] in group[3]:
                associated_groups.append(group)
    
    return associated_groups

## Execute

This is where the execution takes place.

In [5]:
if __name__ == "__main__":
    # create instance of argparse
    parser = argparse.ArgumentParser()

    # Add an argument
    parser.add_argument('--prefix', type=str, required=True)

    # Parse the argument
    args = parser.parse_args()

    # Print "Hello" + the user input argument
    print('Hello,', args.prefix)

    # pull in our configuration objects
    address_groups, address_objects = grab_config()

    # take in our search parameter in the form of a string
    # prefix = "10.232.210.0/26"

    # pass them into the main function
    associated_groups = main(address_groups, address_objects, prefix)
    df = pd.DataFrame(associated_groups)
    print(tabulate(df, headers='keys', tablefmt='psql'))


usage: ipykernel_launcher.py [-h] --prefix PREFIX
ipykernel_launcher.py: error: the following arguments are required: --prefix


SystemExit: 2

/home/cdot/.cache/pypoetry/virtualenvs/pan-python-examples-HGigl0lv-py3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
